In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'1Qywia1SPxzzlTt8nloZbM8R5l_GNql66'}) # replace the id with id of file you want to access
downloaded.GetContentFile('sms_spam_detection.csv') 

In [ ]:
import pandas as pd, nltk
from gensim.models import KeyedVectors, Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-01-30 19:07:50--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.105.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.105.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  47.8MB/s    in 33s     

2021-01-30 19:08:24 (47.0 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
model = KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

# Demovid13 - Document Vector Representation using Word Embeddings
* Extracting vector representation of all the words in corpus
* Creating vector representation for a single document
* Creating vector representation for the entire corpus

In [ ]:
import re, nltk, pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors, Word2Vec
from sklearn.svm import SVC

data = pd.read_csv('sms_spam_detection.csv', encoding='Latin')
data.head()

,target,sms_text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.shape

(5572, 2)

In [ ]:
 'computer' in model

True

In [ ]:
vectorizer = CountVectorizer(lowercase=False).fit(data['sms_text'])
vocab = vectorizer.get_feature_names() #this fn will be able to identify individual tokens in the doc & extract token names and is faster than a for loop

len(vocab)


10800

In [ ]:
print(vocab)

['00', '000', '000pes', '008704050406', '0089', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '078498', '07880867867', '0789xxxxxxx', '07946746291', '0796XXXXXX', '07973788240', '07XXXXXXXXX', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986030', '08002986906', '08002988890', '08006344447', '0808', '08081263000', '08081560665', '0825', '083', '0844', '08448350055', '08448714184', '0845', '08450542832', '08452810071', '08452810073', '08452810075over18', '0870', '0870

In [ ]:
tokens_emb = {}
tokens_missing = []

for x in vocab:
  if x in model:
    tokens_emb[x] = model[x]
  elif x.lower() in model:
    tokens_emb[x] = model[x.lower()]
  else:
    tokens_missing.append(x)

In [ ]:
len(tokens_emb)

8421

In [ ]:
  len(tokens_emb)/len(vocab) * 100

77.97222222222221

In [ ]:
tokens_missing

['00',
 '000',
 '000pes',
 '008704050406',
 '0089',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '050703',
 '0578',
 '06',
 '07',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07781482378',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796XXXXXX',
 '07973788240',
 '07XXXXXXXXX',
 '07xxxxxxxxx',
 '08',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '083',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '084

In [ ]:
doc = data['sms_text'].loc[0]
print(doc)

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


In [ ]:
doc_tokens = doc.split(' ')
doc_tokens[:5]

['Go', 'until', 'jurong', 'point,', 'crazy..']

In [ ]:
doc_tokens_vectors = []
for x in doc_tokens:
  if x in tokens_emb:
    doc_tokens_vectors.append(tokens_emb[x])
doc_tokens_vectors = np.array(doc_tokens_vectors)
doc_tokens_vectors.shape

(12, 300)

In [ ]:
pd.DataFrame(doc_tokens_vectors).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.091309,0.131836,0.019653,0.052490,0.082031,0.038086,0.006500,-0.164062,0.077148,-0.055420,0.047363,0.090820,0.005188,0.006378,-0.249023,-0.162109,0.046143,-0.178711,0.029907,0.156250,-0.026367,0.322266,0.218750,0.166016,-0.159180,0.263672,-0.048340,0.218750,-0.036865,-0.040039,0.044678,0.281250,-0.253906,-0.324219,-0.075684,-0.185547,0.132812,-0.003632,0.058105,-0.007324,...,-0.044678,0.056885,0.097656,0.037598,-0.151367,0.127930,-0.031250,0.099609,-0.053955,0.241211,-0.090820,0.188477,-0.055176,0.183594,0.251953,0.179688,-0.247070,-0.294922,-0.081543,0.054443,-0.026733,-0.085938,-0.214844,0.171875,-0.195312,0.169922,-0.161133,0.369141,0.009216,0.287109,0.087891,0.122070,-0.179688,-0.093750,0.129883,-0.150391,0.202148,-0.361328,0.044189,0.072754
1,0.002380,0.002014,-0.026123,0.106934,0.046143,-0.116699,0.050537,-0.180664,0.275391,0.009827,-0.112793,-0.136719,-0.128906,-0.060791,-0.255859,0.043457,0.328125,0.007599,0.240234,-0.198242,-0.210938,-0.101562,0.146484,0.109863,0.117676,0.062256,0.001869,0.145508,0.229492,0.046143,-0.037354,0.043457,-0.019165,-0.121094,-0.072754,0.042480,-0.073242,-0.251953,-0.011902,-0.070801,...,0.019775,0.044434,-0.253906,0.146484,-0.064941,-0.072266,-0.125000,0.088867,0.136719,-0.050293,0.570312,0.037109,0.159180,0.032227,-0.058350,-0.229492,-0.099121,-0.191406,-0.068359,0.030273,0.201172,-0.038086,0.029663,0.255859,0.140625,-0.084961,0.104980,-0.021973,0.127930,0.186523,-0.058105,0.132812,-0.048340,0.026855,0.060547,0.122559,-0.226562,-0.041260,-0.000675,-0.146484
2,0.100586,-0.191406,-0.462891,0.251953,-0.016235,0.134766,-0.116211,-0.084473,0.376953,0.259766,-0.337891,0.141602,-0.478516,-0.291016,-0.078613,0.111328,0.178711,0.472656,0.097168,-0.281250,-0.235352,0.043701,-0.390625,0.371094,-0.316406,0.226562,-0.103027,0.108398,-0.064941,-0.210938,-0.062012,0.085938,-0.300781,-0.129883,0.080566,-0.174805,-0.247070,-0.398438,-0.171875,-0.086914,...,-0.033691,-0.123535,0.187500,-0.114258,-0.104492,-0.144531,0.386719,-0.032227,-0.049805,-0.308594,0.324219,0.244141,0.063477,0.055664,0.470703,0.209961,-0.032227,-0.145508,0.013245,0.189453,-0.507812,-0.164062,0.300781,-0.093262,0.008850,-0.127930,0.125000,0.423828,0.104980,-0.029785,-0.023804,-0.218750,0.079102,0.404297,0.240234,0.035645,-0.066406,-0.180664,0.281250,0.275391
3,0.093750,-0.093262,0.054443,0.002014,0.003830,0.067383,0.018066,-0.189453,0.143555,0.021729,-0.012695,-0.121582,-0.059326,0.066895,-0.119629,0.034668,0.133789,-0.042236,0.002258,-0.039307,-0.064941,0.023193,-0.138672,-0.021362,0.028564,-0.048584,-0.178711,0.152344,0.021973,0.051025,-0.078125,0.015625,0.043457,0.029785,0.027832,-0.089844,-0.055664,-0.023438,-0.064453,-0.013794,...,-0.003387,0.001740,-0.127930,0.014404,0.031250,-0.004700,0.020386,0.028442,-0.064941,-0.007263,0.141602,-0.049072,0.130859,0.072266,0.038818,-0.166992,-0.147461,-0.157227,-0.054932,-0.015747,0.071289,-0.029053,0.108398,0.049072,-0.060303,-0.040283,-0.128906,-0.097656,-0.113281,0.097168,-0.041748,-0.046631,0.007446,-0.073242,0.021729,0.142578,-0.067871,-0.092773,-0.024536,-0.084961
4,0.070312,0.086914,0.087891,0.062500,0.069336,-0.108887,-0.081543,-0.154297,0.020752,0.131836,-0.113770,-0.037354,0.069336,0.078125,-0.103027,-0.097656,0.044189,0.102539,-0.060791,-0.036133,-0.045410,0.047363,-0.120605,-0.063965,0.002258,0.037109,-0.002914,0.117676,0.061768,0.063965,0.081055,-0.068848,-0.021362,0.055176,-0.085449,0.068848,-0.127930,-0.033203,0.098633,0.175781,...,0.004730,0.020508,0.071289,0.170898,0.173828,0.055664,0.091309,-0.037354,0.049805,-0.039307,0.044189,0.062500,0.048584,-0.053223,0.048828,-0.130859,-0.028931,-0.036133,-0.060791,-0.057373,0.123047,-0.082520,-0.011902,0.125000,0.001358,0.063965,-0.106445,-0.143555,-0.042236,0.02

* Above is a 2D representation of doc
* We need to covert 2D to 1D via means
* ML alogrithms requries input in 1D form compared to neural networks which can take input in many dimensions(Matrix like)

In [ ]:
doc_vector = doc_tokens_vectors.mean(axis=0)
len(doc_vector)

300

* This is 1D representation of above 2D doc

In [ ]:
def get_doc_vector(doc, tokens_emb):
  doc_tokens_vectors = []
  for x in doc.split(' '):
    if x in tokens_emb:
      doc_tokens_vectors.append(tokens_emb[x])
  doc_tokens_vectors = np.array(doc_tokens_vectors)
  doc_vector1 = doc_tokens_vectors.mean(axis=0)
  if doc_vector1.shape == ():
    doc_vector1 = np.zeros(300)
  return doc_vector1


* There can be some words which don't have vector representation 
* To handle such cases we check the shape of the doc " if doc_vector.shape == () :"
* If it's empty, we are creating a manual vector represntation which will have all zeros via "doc_vector = np.zeros(300)"
      



In [ ]:
docs_vectors1 = []

for x in data['sms_text']:
  doc_vector = get_doc_vector(doc, tokens_emb)
  docs_vectors1.append(doc_vector)


In [ ]:
docs_vectors1 = np.array(docs_vectors1)
docs_vectors1.shape

(5572, 300)

In [ ]:
pd.DataFrame(docs_vectors1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.049515,0.037181,-0.012146,0.13003,0.016382,0.013652,0.035372,-0.139099,0.069778,0.109126,-0.116964,-0.102437,-0.039876,-0.012703,-0.104655,0.075541,0.167643,0.14254,0.007238,-0.028554,-0.110668,0.045425,0.056981,0.042572,0.018967,0.020549,-0.094183,0.06367,0.042704,-0.015991,-0.022374,0.044207,-0.086711,-0.060272,0.001424,-0.060072,-0.021891,-0.03868,0.004584,0.016408,...,-0.050527,0.014936,0.035196,-0.009165,0.002385,0.002864,0.051153,-0.003733,-0.036306,0.024465,0.032552,0.081034,0.053991,0.007116,0.156169,0.011734,-0.143016,-0.110291,-0.0725,0.06721,-0.072011,-0.031006,-0.026388,0.051737,0.021023,0.042358,-0.02491,0.012103,0.003555,0.073751,-0.073629,-0.007207,-0.111196,0.081482,0.061727,0.028422,0.004842,-0.082397,0.012898,0.000773
1,0.049515,0.037181,-0.012146,0.13003,0.016382,0.013652,0.035372,-0.139099,0.069778,0.109126,-0.116964,-0.102437,-0.039876,-0.012703,-0.104655,0.075541,0.167643,0.14254,0.007238,-0.028554,-0.110668,0.045425,0.056981,0.042572,0.018967,0.020549,-0.094183,0.06367,0.042704,-0.015991,-0.022374,0.044207,-0.086711,-0.060272,0.001424,-0.060072,-0.021891,-0.03868,0.004584,0.016408,...,-0.050527,0.014936,0.035196,-0.009165,0.002385,0.002864,0.051153,-0.003733,-0.036306,0.024465,0.032552,0.081034,0.053991,0.007116,0.156169,0.011734,-0.143016,-0.110291,-0.0725,0.06721,-0.072011,-0.031006,-0.026388,0.051737,0.021023,0.042358,-0.02491,0.012103,0.003555,0.073751,-0.073629,-0.007207,-0.111196,0.081482,0.061727,0.028422,0.004842,-0.082397,0.012898,0.000773
2,0.049515,0.037181,-0.012146,0.13003,0.016382,0.013652,0.035372,-0.139099,0.069778,0.109126,-0.116964,-0.102437,-0.039876,-0.012703,-0.104655,0.075541,0.167643,0.14254,0.007238,-0.028554,-0.110668,0.045425,0.056981,0.042572,0.018967,0.020549,-0.094183,0.06367,0.042704,-0.015991,-0.022374,0.044207,-0.086711,-0.060272,0.001424,-0.060072,-0.021891,-0.03868,0.004584,0.016408,...,-0.050527,0.014936,0.035196,-0.009165,0.002385,0.002864,0.051153,-0.003733,-0.036306,0.024465,0.032552,0.081034,0.053991,0.007116,0.156169,0.011734,-0.143016,-0.110291,-0.0725,0.06721,-0.072011,-0.031006,-0.026388,0.051737,0.021023,0.042358,-0.02491,0.012103,0.003555,0.073751,-0.073629,-0.007207,-0.111196,0.081482,0.061727,0.028422,0.004842,-0.082397,0.012898,0.000773
3,0.049515,0.037181,-0.012146,0.13003,0.016382,0.013652,0.035372,-0.139099,0.069778,0.109126,-0.116964,-0.102437,-0.039876,-0.012703,-0.104655,0.075541,0.167643,0.14254,0.007238,-0.028554,-0.110668,0.045425,0.056981,0.042572,0.018967,0.020549,-0.094183,0.06367,0.042704,-0.015991,-0.022374,0.044207,-0.086711,-0.060272,0.001424,-0.060072,-0.021891,-0.03868,0.004584,0.016408,...,-0.050527,0.014936,0.035196,-0.009165,0.002385,0.002864,0.051153,-0.003733,-0.036306,0.024465,0.032552,0.081034,0.053991,0.007116,0.156169,0.011734,-0.143016,-0.110291,-0.0725,0.06721,-0.072011,-0.031006,-0.026388,0.051737,0.021023,0.042358,-0.02491,0.012103,0.003555,0.073751,-0.073629,-0.007207,-0.111196,0.081482,0.061727,0.028422,0.004842,-0.082397,0.012898,0.000773
4,0.049515,0.037181,-0.012146,0.13003,0.016382,0.013652,0.035372,-0.139099,0.069778,0.109126,-0.116964,-0.102437,-0.039876,-0.012703,-0.104655,0.075541,0.167643,0.14254,0.007238,-0.028554,-0.110668,0.045425,0.056981,0.042572,0.018967,0.020549,-0.094183,0.06367,0.042704,-0.015991,-0.022374,0.044207,-0.086711,-0.060272,0.001424,-0.060072,-0.021891,-0.03868,0.004584,0.016408,...,-0.050527,0.014936,0.035196,-0.009165,0.002385,0.002864,0.051153,-0.003733,-0.036306,0.024465,0.032552,0.081034,0.053991,0.007116,0.156169,0.011734,-0.143016,-0.110291,-0.0725,0.06721,-0.072011,-0.031006,-0.026388,0.051737,0.021023,0.042358,-0.02491,0.012103,0.003555,0.073751,-0.073629,-0.007207,-0.111196,0.081482,0.061727,0.028422,0.004842,-0.